In [1]:
import pandas as pd
import random

In [2]:
# World Cup Matches
matches = pd.read_csv("../WorldCupMatches.csv")

# World Cup Players
players = pd.read_csv("../WorldCupPlayers.csv")

# World Cups
cups = pd.read_csv("../WorldCups.csv")

# Player Stats
stats = pd.read_csv("../PlayersStats.csv")

# Merge Players with Matches

# Add Player's overall FIFA score

In [3]:
cols = matches.columns
cols = list(cols)

new = [
    "Home Coach Name",
    "Home Player 1", "Home Player 2", 
    "Home Player 3", "Home Player 4", 
    "Home Player 5", "Home Player 6", 
    "Home Player 7", "Home Player 8", 
    "Home Player 9", "Home Player 10",
    "Home Player 11",

    "Away Coach Name",
    "Away Player 1", "Away Player 2", 
    "Away Player 3", "Away Player 4", 
    "Away Player 5", "Away Player 6", 
    "Away Player 7", "Away Player 8", 
    "Away Player 9","Away Player 10",
    "Away Player 11",
    
    "Home Player 1 Stats", "Home Player 2 Stats", 
    "Home Player 3 Stats", "Home Player 4 Stats", 
    "Home Player 5 Stats", "Home Player 6 Stats", 
    "Home Player 7 Stats", "Home Player 8 Stats", 
    "Home Player 9 Stats","Home Player 10 Stats",
    "Home Player 11 Stats",

    "Away Player 1 Stats", "Away Player 2 Stats", 
    "Away Player 3 Stats", "Away Player 4 Stats", 
    "Away Player 5 Stats", "Away Player 6 Stats", 
    "Away Player 7 Stats", "Away Player 8 Stats", 
    "Away Player 9 Stats","Away Player 10 Stats",
    "Away Player 11 Stats"
]

cols += new
final = pd.DataFrame(columns=cols)

### Merging (will take about an hour)

In [81]:
DEFAULT_OVERALL_SCORE = 65

for i in range(len(matches)):
    home = ["coach"]
    away = ["coach"]
    HomeInitials = matches.iloc[i]["Home Team Initials"]
    AwayInitials = matches.iloc[i]["Away Team Initials"]   

    for j in range(len(players)):
        player = players.iloc[j]["Player Name"]
        coach = players.iloc[j]["Coach Name"]
            
        if HomeInitials == players.iloc[j]["Team Initials"]:
            home[0] = coach
            home.append(player)
        elif AwayInitials == players.iloc[j]["Team Initials"]:
            away[0] = coach
            away.append(player)
        else:
            pass
        
    while len(home) != 12:
        if len(home) > 12:
            del home[random.randint(1, len(home) - 1)]
        else:
            home.append(random.choice(home))
        
        
    while len(away) != 12:
        if len(away) > 12:
            del away[random.randint(1, len(away) - 1)]
        else:
            away.append(random.choice(away))
        

    # Add Player's overall FIFA score
    
    home_stats = []
    for x in home[1:]:
        name = str(x).lower().strip()
        
        for j in range(len(stats)):            
            current_name = stats.iloc[j]["Name"]
            current_name = str(current_name).lower().strip()
            
            try:
                if name in current_name:
                    # print("1", name, " | ", current_name)
                    home_stats.append(stats.iloc[j]["Overall"])
                    break

                elif name.split(" ")[1] in current_name:
                    # print("2", name, " | ", current_name)
                    home_stats.append(stats.iloc[j]["Overall"])
                    break

                elif name.split(" ")[0] in current_name:
                    # print("3", name, " | ", current_name)
                    home_stats.append(stats.iloc[j]["Overall"])
                    break

                elif name.split(" ")[2] in current_name:
                    # print("4", name, " | ", current_name)
                    home_stats.append(stats.iloc[j]["Overall"])
                    break

                else:
                    pass
            except:
                pass
            
            if j == len(stats)-1:
                home_stats.append(DEFAULT_OVERALL_SCORE)
    
    away_stats = []
    for x in away[1:]:
        name = str(x).lower().strip()
        
        for j in range(len(stats)):            
            current_name = stats.iloc[j]["Name"]
            current_name = str(current_name).lower().strip()
            
            try:
                if name in current_name:
                    # print("1", name, " | ", current_name)
                    away_stats.append(stats.iloc[j]["Overall"])
                    break

                elif name.split(" ")[1] in current_name:
                    # print("2", name, " | ", current_name)
                    away_stats.append(stats.iloc[j]["Overall"])
                    break

                elif name.split(" ")[0] in current_name:
                    # print("3", name, " | ", current_name)
                    away_stats.append(stats.iloc[j]["Overall"])
                    break

                elif name.split(" ")[2] in current_name:
                    # print("4", name, " | ", current_name)
                    away_stats.append(stats.iloc[j]["Overall"])
                    break

                else:
                    pass
            except:
                pass
            
            if j == len(stats)-1:
                away_stats.append(DEFAULT_OVERALL_SCORE)
    
    data = list(matches.iloc[i])
    data += home
    data += away
    data += home_stats
    data += away_stats
    
    final = final.append(pd.Series(data, index=cols), ignore_index=True)

In [83]:
final.head()

,Year,Datetime,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Attendance,Half-time Home Goals,Half-time Away Goals,...,Away Player 2 Stats,Away Player 3 Stats,Away Player 4 Stats,Away Player 5 Stats,Away Player 6 Stats,Away Player 7 Stats,Away Player 8 Stats,Away Player 9 Stats,Away Player 10 Stats,Away Player 11 Stats
0,2010.0,11 Jun 2010 - 16:00,Group A,South Africa,1.0,1.0,Mexico,84490.0,0.0,0.0,...,80,63,84,76,81,83,91,90,94,94
1,2010.0,11 Jun 2010 - 20:30,Group A,Uruguay,0.0,0.0,France,64100.0,0.0,0.0,...,76,65,64,71,80,89,65,65,65,81
2,2010.0,12 Jun 2010 - 13:30,Group B,Korea Republic,2.0,0.0,Greece,31513.0,1.0,0.0,...,63,65,82,71,63,71,81,63,65,65
3,2010.0,12 Jun 2010 - 16:00,Group B,Argentina,1.0,0.0,Nigeria,55686.0,1.0,0.0,...,76,80,74,65,76,74,74,74,74,74
4,2010.0,12 Jun 2010 - 20:30,Group C,England,1.0,1.0,USA,38646.0,1.0,1.0,...,71,65,80,72,65,70,74,78,71,71


# Save final into CSV file

In [82]:
final.to_csv("../final.csv", index=0)